# PCA from Scratch

## Motivation

PCA  i.e. Principal Component Analysis is a dimension reduction method commonly used in machine learning. It is commonly used for reducing the number of columns when there is thousands of columns whcih becomes exponentially diificult to manage with their increasing number. The naive idea to reduce column numbers is to cut off the columns but then we will have massive loss of information. Bur here PCA ensuers compact featurs(columns) with minimum loss of information.

The main idea of PCA is to find a component onto which projection of data have maximum variance. For example if we want to reduce two columns $\,x_1 \,, x_2\,$ in one then we need to plot each entry(blue dots) in a cartesian system with $x_1$ as abscissa and $x_2$ as ordiante where we try to find the best possible line(component) where the values have maximum variance while projected.<br>
<div>
    <center>
        <img src="pca-example-1D-of-2D.png" width=300>
    </center>
</div>
<br>
<center> Apparantly the line $u_1$ seems to quite fit for our purpose. Where each value $x^i$ is projected as $\tilde{x}^i$ on $u_1$ with maximum variance. </center>


## Perfroming PCA

### Step-1: Get The Covariance matirx


First we get the covariance matrix for all the features in question.
For any two variabel $x$ and $y$ we use this formula for covariance
### $$cov_{x,y}=\frac{\sum(x_{i}-\bar{x})(y_{i}-\bar{y})}{N-1}$$
<center>
    $cov_{x,y}$ = covariance between variable x and y <br>
    $x_{i}$ = data value of x <br>
    $y_{i}$ = data value of y <br>
    $\bar{x}$ = mean of x <br>
    $\bar{y}$ = mean of y <br>
    $N$ = number of data values
</center>
<br>
And a covarinace matrix is a $m \times m\,$ square matrix where any element $x_{ij}\,$ is an element at row $i$ and column $j$ is covariance of features $\,X_i\,$ and $\,X_j$

#### $$ \Sigma = \begin{bmatrix}Var(X_1) & Cov(X_1, X_2) & \ldots & Cov(X_1, X_{m}) \\
                          Cov(X_1, X_2) & Var(X_2) & \ldots & Cov(X_2, X_{m})\\
                          \vdots & \vdots & \ddots & \vdots \\
                          Cov(X_1,X_{m}) & Cov(X_2, X_{m}) &\ldots & Var(X_{m})\end{bmatrix}$$

It turns out if we have a matrix $X$ of the data with the column of tha mtrix being each featue we can perform multiplication of $X^T$ and $X$ and divide every element with $N-1$ where $N$ is the number of observations.

### Step-2: Get Eigenvectors of Covariance Matrix

Next we get the eigenvectors of the covariance matrix. For this we are going to use the Singular Value Decompositio or SVD. SVD of a matrix $X$ looks like this

$$ X = U \Sigma V^T $$
Where $U$ and $V$ are two orthonormal matrices and $\Sigma$ is a diagonal matrix of singular values of $X$.<br>
<br>
SVD of a square matrix acts as eigen decomposition which yields two matrices $P$ and $D$,
$$ X = PDP^{-1} $$
Where $ U = V = P$ and $ \Sigma = D$. Here orthonormal matrices $P$ is the collection of unit eigenvectors of matrices $X$ and $D$ is a diagonal matrices with eigen values at its diagonal. The eigenvectors serve as the principal components where we can project our values.

### Step-3: Project the Values onto the Principal Components

Projection of any vector $x$ onto vector $y$ is given by,
#### $$ proj_y(x) = \frac{\langle x,y \rangle}{||y||^2} $$
As we are going to project onto unit eigenvectors the norm of $y$ is $1$ or $||y||^2 = 1$. Which eases our calculations of $proj_y(x)$ to,
#### $$ proj_y(x) = \langle x,y \rangle \,  $$
Inner product of $x$ and $y$ can be written as matrix multiplication of $x^T$ and $y$,
#### $$ proj_y(x) = \langle x,y \rangle = x^T y = \begin{bmatrix}\ldots &x& \ldots \end{bmatrix} \, \begin{bmatrix}\vdots \\ y \\ \vdots \end{bmatrix} $$
We need to project all the entries of a data stored in matrix $X$ on the principal component.
#### $$ proj_y(X) = \begin{bmatrix}\ldots &x_1& \ldots \\ \ldots &x_2& \ldots \\ \vdots&\vdots&\vdots \\ \ldots &x_n& \ldots \end{bmatrix} \, \begin{bmatrix}\vdots \\ y \\ \vdots \end{bmatrix} = X^T\begin{bmatrix}\vdots \\ y \\ \vdots \end{bmatrix}$$
If there is several components then we project in all components which are contained in matrices $Y$
#### $$ proj_y(X) = X^T \begin{bmatrix} \vdots&\vdots&\vdots&\vdots \\ y_1&y_2&\ldots&y_n \\ \vdots&\vdots&\vdots&\vdots \end{bmatrix} = X^TY $$

And in the end the $\,proj_y(X)\,$ is our reduced dimension or columns.

### Let's Code PCA

It's time we code up all the theories we have learned so far. <br>
In order to implement Our PCA we will,<br>
1. Calculate covariance matrix with the function `cov()` from `mystats`
2. Calculate eigenvectors of covariance matrix by `svd()` from `mylinalg`
3. Project each entry(row) onto the defined number of eigenvectros

N.B. `cov()` from `mystats` is coded from scratch depending on the idea that $cov(X) = \frac{X^TX}{N-1}$

N.B. `matmul` and `svd()` from `mylinalg` is coded from scratch without help of any advance library. In order to undersatnd the implemntation of `svd()` please visit my linkedin post <a href="http://linekedin.com" >Things I learnded while implementing PCA from scratch</a>

In [15]:
from mystats import cov
from mylinalg import svd, matmul

In [16]:
def myPCA(data, n_components=1):
    cov_mat = cov(data)
    U, S, Vh = svd(cov_mat)
    components = [U[i][:n_components] for i in range(len(U))]
    reduced = matmul(data, components)
    return reduced

Let's compare our `myPCA` with `PCA` from sklearn

In [18]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [19]:
cancer = load_breast_cancer()
df = pd.DataFrame(cancer["data"], columns=cancer["feature_names"])
scaled_data = StandardScaler().fit_transform(df)

In [20]:
# Number of components we want to reduce to
n_components = 3

In [21]:
pca = PCA(n_components=n_components)
pca.fit(scaled_data)
x_pca = pca.transform(scaled_data)

In [22]:
my_pca = myPCA(scaled_data.tolist(), n_components=n_components)

In [23]:
print("Reduction performed by 'sklearn.decomposition.PCA'")
print(x_pca)
print('')
print("Reduction performed by our from scratch 'myPca'")
for r in my_pca[:3]:
    print(r)
print("...")
for r in my_pca[-3:]:
    print(r)

Reduction performed by 'sklearn.decomposition.PCA'
[[ 9.19283683  1.94858308 -1.12316635]
 [ 2.3878018  -3.76817174 -0.52929295]
 [ 5.73389628 -1.0751738  -0.55174759]
 ...
 [ 1.25617928 -1.90229672  0.56273061]
 [10.37479406  1.6720101  -1.87702921]
 [-5.4752433  -0.67063678  1.49044265]]

Reduction performed by our from scratch 'myPca'
[-9.192837666695365, 1.948559386649288, -1.123162809958965]
[-2.387800170472775, -3.768178337938722, -0.5292583364121393]
[-5.7338958159750355, -1.0751882951008802, -0.5517348127949726]
...
[-1.2561784556313793, -1.90229861743443, 0.5627345368981741]
[-10.374794780956703, 1.6719822147568666, -1.877053893158748]
[5.475243591808819, -0.6706211475603386, 1.4904366899479098]


#### signs may be opposite for corresponding column due the reversal of eigenvector. Nonetheless the values work as expected